<h2>[ Preview ]</h2>

<h4>
<ul>
    <li>Naive Bayes Classification</li>
    <li></li>
    <li></li>
    <li></li>
</ul>
</h4>

<h2>[ Naive Bayes Classification ]</h2>

<h3>1. 나이브 베이즈 분류</h3>
<h4>
    <ul>
        <li>스팸 메일 필터, 텍스트 분류, 감정 분석, 추천 시스템 등에 광범위하게 활용되는 분류 기법</li>
        <li>Feature에 따라 Label을 분류 => 모든 Feature가 서로 독립(independent)이어야 한다는 가정이 필요</li>
    </ul>
</h4>



<h3>2. 지도학습 - 나이브 베이즈 분류 방법</h3>
<h5>
    <ul>
        <li>다양한 자세, 표정, 생김새, 털의 색깔 등을 가진 호랑이, 고양이, 얼룩말에 대해 '이 사진은 호랑이고, 이 사진은 고양이야'라고 학습시키는 것</li>
        <li> 학습시 사용되었던 사진이 아닌 새로운 사진을 갖다 줬을 때도 정확히 분류</li>
        <li>사전 데이터를 기반으로 충분히 학습시키는 방법을 지도학습(Supervised learning)</li>
        <li>지도학습을 하기 위한 첫 스텝은 Feature와 Label을 파악하는 것</li>
        <li>Label : 우리가 원하는 분류 결과 => 호랑이, 고양이, 얼룩말</li>
        <li>Feature : Label 결과에 영항을 주는 요소 => 자세, 표정, 생김새, 털</li>
    </ul>
</h5>

<h3>3. 나이브 베이즈 동작</h3>
<img src="../Image/CH14/Naive_1.JPG" width="600" height="400" title=""/>

<img alig="left" src="../Image/CH14/Naive_2.JPG" width="200" height="400" title=""/>
<img alig="right" src="../Image/CH14/Naive_3.JPG" width="600" height="400" title=""/>


---

<h3>1. 데이터 전처리</h3>

In [32]:
import pandas as pd

df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


<h4>1-1. 변수 삭제</h4>
<h5>의미 없는 컬럼을 지우기</h5>

In [33]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'], axis='columns', inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [34]:
# target과 input 나누기

target = df.Survived
inputs = df.drop(['Survived'], axis='columns')
inputs.head()

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500


In [35]:
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


<h4>1-2. get_dummies() 사용해서 열 변환</h4>
<h5>해당 열의 값에 따라 DataFrame 생성</h5>

In [36]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head()

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1


<h4>1-3. DataFrame 합치기 => pd.concat()</h4>
<h5>Sex 컬럼의 원소인 male, female가 열인 DF를 합치기</h5>

In [37]:
inputs = pd.concat([inputs, dummies], axis= 'columns')
inputs.head()

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0
3,1,female,35.0,53.1000,1,0
4,3,male,35.0,8.0500,0,1


In [38]:
inputs.drop(['Sex'], axis='columns', inplace=True)
inputs.head()

,Pclass,Age,Fare,female,male
0,3,22.0,7.2500,0,1
1,1,38.0,71.2833,1,0
2,3,26.0,7.9250,1,0
3,1,35.0,53.1000,1,0
4,3,35.0,8.0500,0,1


<h4>1-4. 결측치 확인 => isna()</h4>
<h5>결측치를 평균 값으로 채우기</h5>

In [39]:
inputs.columns

Index(['Pclass', 'Age', 'Fare', 'female', 'male'], dtype='object')

In [40]:
# 결측치가 Age 컬럼에 있음

inputs.isna().any()

Pclass    False
Age        True
Fare      False
female    False
male      False
dtype: bool

In [41]:
# 결측치가 있는 컬럼 추출

inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [42]:
# Age 컬럼 결측치를 평균 값으로 채우기

inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.isna().any()

Pclass    False
Age       False
Fare      False
female    False
male      False
dtype: bool

---

<h3>2. 모델 훈련</h3>
<h4>2-1. train / test data 분리 </h4>

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size= 0.2)

In [47]:
len(X_train), len(X_test), len(y_train), len(y_test)

(712, 179, 712, 179)

<h4>2-2. Naive Bayes Model </h4>

In [48]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [49]:
model.fit(X_train, y_train)

GaussianNB()

In [51]:
model.score(X_test, y_test)

0.7877094972067039

<h4>2-3. Naive Bayes Model 확인하기 </h4>

In [52]:
X_test[:10]

,Pclass,Age,Fare,female,male
252,1,62.000000,26.5500,0,1
309,1,30.000000,56.9292,1,0
838,3,32.000000,56.4958,0,1
700,1,18.000000,227.5250,1,0
639,3,29.699118,16.1000,0,1
704,3,26.000000,7.8542,0,1
234,2,24.000000,10.5000,0,1
85,3,33.000000,15.8500,1,0
468,3,29.699118,7.7250,0,1
569,3,32.000000,7.8542,0,1


In [53]:
y_test[:10]

252    0
309    1
838    1
700    1
639    0
704    0
234    0
85     1
468    0
569    1
Name: Survived, dtype: int64

In [54]:
# 위의 y_test와 비교하면 조금 다름

model.predict(X_test[:10])

array([0, 1, 0, 1, 0, 0, 0, 1, 0, 0], dtype=int64)

In [56]:
# 각각의 확률 도시화

model.predict_proba(X_test[:10])

array([[8.84651120e-01, 1.15348880e-01],
       [4.32105422e-03, 9.95678946e-01],
       [9.82337710e-01, 1.76622904e-02],
       [5.49266843e-10, 9.99999999e-01],
       [9.90388419e-01, 9.61158051e-03],
       [9.89889691e-01, 1.01103094e-02],
       [9.76416613e-01, 2.35833874e-02],
       [6.55122782e-02, 9.34487722e-01],
       [9.90225390e-01, 9.77460982e-03],
       [9.90356483e-01, 9.64351727e-03]])

In [1]:
# K-Fold 사용하기

from sklearn.model_selection import cross_val_score

cross_val_score(GaussianNB(), X_train, y_train, cv=5)

NameError: name 'GaussianNB' is not defined

---
---

In [23]:
import pandas as pd

df = pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


<h3>1. 데이터 전처리</h3>

In [24]:
df.Category.unique()

array(['ham', 'spam'], dtype=object)

In [25]:
df.describe()

,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


In [26]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

<h4>1-1. 텍스트 값을 숫자로 변환</h4>

In [27]:
df['spam'] = df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


---

<h3>2. 모델 훈련</h3>
<h4>2-1. train / test data 분리 </h4>

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.Message, df.spam, test_size=0.25)

<h4>2-2. CountVectorizer</h4>
<h6>문자열로 되어있는 원소를 숫자로 어떻게 변환할까</h6>

<h5>
<ul>
    <li>단어들의 카운트(출현 빈도(frequency))로 여러 문서들을 벡터화</li>
    <li>모두 소문자로 변환시키기 때문에 me 와 Me 는 모두 같은 특성</li>
    <li>단위는 정하는대로 => 가장 많이 사용되는 것은 단어단위의 카운팅</li>
    <li>단어의 출현 빈도를 사용하여, 어떤 단어가 많이 발생하는 것에 대해서 어떠한 특징을 지닌다는 법칙을 발견하는 머신러닝 알고리즘의 입력값으로 사용</li>
    <li>문제 : 특별한 의미를 지니지 않는데 자주 사용되는 단어는 카운팅에서 높은 횟수를 가지게 되지만, 실질적으론 의미가 없는 데이터라서 별로 좋지 않은 형태</li>
    <li>딥러닝이라면, 그냥 이 카운팅을 사용해도 상관은 없습니다. 해당 노드에서 들어오는 가중치를 낮춤으로써 결과에의 영향력을 스스로 줄이지만 좋은 모델을 위해 해결이 필요 => TF-IDF</li>
</ul>
</h5>

<img src="../Image/CH14/Naive_4.JPG" width="600" height="400" title=""/>

<h4>TF-IDF</h4>
<h5>
<ul>
    <li>Term Frequency – Inverse Document Frequency</li>
    <li>인코딩은 단어를 갯수 그대로 카운트하지 않고 모든 문서에 공통적으로 들어있는 단어의 경우 문서 구별 능력이 떨어진다고 보아 가중치를 축소하는 방법</li>

</ul>
</h5>

In [11]:
X_train.values

array(["I'm done. I'm sorry. I hope your next space gives you everything you want. Remember all the furniture is yours. If i'm not around when you move it, just lock all the locks and leave the key with jenne.",
       'Buy Space Invaders 4 a chance 2 win orig Arcade Game console. Press 0 for Games Arcade (std WAP charge) See o2.co.uk/games 4 Terms + settings. No purchase',
       'Miles and smiles r made frm same letters but do u know d difference..? smile on ur face keeps me happy even though I am miles away from u.. :-)keep smiling.. Good nyt',
       ...,
       '3 FREE TAROT TEXTS! Find out about your love life now! TRY 3 FOR FREE! Text CHANCE to 85555 16 only! After 3 Free, Msgs £1.50 each',
       "Nobody names their penis a girls name this story doesn't add up at all",
       "How's it feel? Mr. Your not my real Valentine just my yo Valentine even tho u hardly play!!"],
      dtype=object)

In [14]:
# CountVectorizer를 이용해 문자열을 숫자로 변환

from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()
X_train_count = v.fit_transform(X_train.values)
X_train_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

<h4>2-3. Multinomial NB</h4>

In [16]:
# 문자열을 숫자로 변환한 데이터를 활용해 모델 적용

from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_count, y_train)

MultinomialNB()

In [17]:
# 문자열을 숫자로 변환한 모델을 이용해 이메일을 숫자로 변환

emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20"%" discount on parking, exclusive offer just for you. Dont miss this reward!'
]
emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 1], dtype=int64)

In [18]:
X_test_count = v.transform(X_test)
model.score(X_test_count, y_test)

0.9806173725771715

<h4>3. Sklearn Pipline</h4>

In [19]:
# API처럼 사용하기 위해 파이프라인 사용(순차적으로 모델에 적용함)

from sklearn.pipeline import Pipeline
clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [20]:
clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [21]:
clf.score(X_test,y_test)

0.9806173725771715

In [22]:
clf.predict(emails)

array([0, 1], dtype=int64)

---

<h4> 참고 : CountVectorizer </h4>

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?',
]
vect = CountVectorizer()
vect.fit(corpus)
vect.vocabulary_

{'this': 9,
 'is': 3,
 'the': 7,
 'first': 2,
 'document': 1,
 'second': 6,
 'and': 0,
 'third': 8,
 'one': 5,
 'last': 4}

In [72]:
vect.transform(['This is the second document.']).toarray()

array([[0, 1, 0, 1, 0, 0, 1, 1, 0, 1]], dtype=int64)

In [74]:
vect.transform(corpus).toarray()

array([[0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]], dtype=int64)

<h4> 참고 : TF_IDF </h4>

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidv = TfidfVectorizer()
tfidv.fit(corpus)
tfidv.transform(corpus).toarray()

array([[0.        , 0.38947624, 0.55775063, 0.4629834 , 0.        ,
        0.        , 0.        , 0.32941651, 0.        , 0.4629834 ],
       [0.        , 0.24151532, 0.        , 0.28709733, 0.        ,
        0.        , 0.85737594, 0.20427211, 0.        , 0.28709733],
       [0.55666851, 0.        , 0.        , 0.        , 0.        ,
        0.55666851, 0.        , 0.26525553, 0.55666851, 0.        ],
       [0.        , 0.38947624, 0.55775063, 0.4629834 , 0.        ,
        0.        , 0.        , 0.32941651, 0.        , 0.4629834 ],
       [0.        , 0.45333103, 0.        , 0.        , 0.80465933,
        0.        , 0.        , 0.38342448, 0.        , 0.        ]])

---